In [7]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pytz
import urllib as ur
import json

## Funções para auxiliar a manipulação do tempo

In [8]:
# Converte hora dada a time zone atual, a zona a ser convertida e a diferença de tempo
def convert_datetime_timezone(dt, tz1, tz2, str_diff):
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)

    dt = datetime.datetime.fromtimestamp(dt)
    dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S")
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    dt = datetime.datetime.strptime(str(dt),"%Y-%m-%d %H:%M:%S"+str_diff)
    dt = int(dt.timestamp())

    return dt

In [9]:
# Faz a diferença entre duas horas dadas e retorna em minutos
def Hour_Diff(h1,h2):
    h1Aux = datetime.datetime.fromtimestamp(h1)
    h2Aux = datetime.datetime.fromtimestamp(h2)
    diff = abs((h1Aux - h2Aux)).total_seconds()/60
    
    return diff

## Requisitando Bairros e seus respectivos estacionamentos, carros e coordenadas

In [10]:
# Request na api para pegar bairros, localizações e ids dos carros

#Formatando a request
response = ur.request.urlopen('https://bookit.modo.coop/api/fleet').read().decode('UTF-8')
json_Neighbourhood = json.loads(response)

response = ur.request.urlopen('https://bookit.modo.coop/api/fleet/locations').read().decode('UTF-8')
json_file = json.loads(response)

NeighbourhoodsIDs = list(json_Neighbourhood['Neighbourhoods'].keys())

location_id = []

for i in NeighbourhoodsIDs:
    for j in list(json_file['Neighbourhoods'][i]['Locations'].keys()):
        location_id.append([i,j])

# Dataframe com bairros e localizações
location_id = pd.DataFrame(location_id, columns=['neighbourhoods', 'locations'])

Cars = []
auxCar = []
IDs = []
auxLoc = []
coord = []

for i in range(len(location_id)):
        neighbourhood = location_id['neighbourhoods'].iloc[i]
        loc = location_id['locations'].iloc[i]
        auxCar.append(list(json_file['Neighbourhoods'][neighbourhood]['Locations'][loc]['Cars']))
        
        # Percorre a lista com as IDs de localizações para pegar as multiplas ocorrências
        lista = list(json_file['Neighbourhoods'][neighbourhood]['Locations'][loc]['Cars'])
        for k in lista:
            auxLoc.append([neighbourhood, loc])
            
        # Armazena as localizações e suas respectivas coordenadas
        coord.append([loc, json_file['Neighbourhoods'][neighbourhood]['Locations'][loc]['Latitude'], 
                      json_file['Neighbourhoods'][neighbourhood]['Locations'][loc]['Longitude']])

#Reformatando para uma lista unidimensionalfor i in auxCar:
for i in auxCar:    
    for j in i:
        Cars.append(j)

Cars = pd.DataFrame(Cars, dtype=int)
location_id = pd.DataFrame(auxLoc, dtype=int)
location_id = pd.concat([location_id, Cars], axis=1)
location_id.columns = ['neighbourhood', 'location', 'car_id']

loc_coord = pd.DataFrame(coord, columns=['location', 'lat', 'lon'])

# Ordenando o dataframe pelas localizações
location_id = location_id.sort_values(by=['location'])

In [11]:
# Separando carros com suas respectivas coordenadas
car_coord = []

for i in range(len(loc_coord)):
    # Pegando todas localizações a partir das localizações salvas no loc_coord
    aux = location_id[location_id['location'] == int(loc_coord['location'].iloc[i])]
    quant = len(aux)
    
    # Pegando as multiplas ocorrências de localizações para ter as coordenadas de cada carro
    for j in range(quant):
        car_coord.append([aux['location'].iloc[j], aux['car_id'].iloc[j], loc_coord['lat'].iloc[i], loc_coord['lon'].iloc[i]])

car_coord = pd.DataFrame(car_coord, columns=['location', 'car_id', 'lat', 'lon'])

## Fazendo a média da quantidade de viagens para cada hora

In [12]:
# CSV criado a partir dos dados coletados do arquivo ModoApi_Data_Filter
dfTravels = pd.read_csv('Travels.csv', usecols=[1,2,3,4])
dfTravels = dfTravels.sort_values(by='car_id')

In [13]:
def cont_travels(df):
    df_cont = []
    id_prox = df['car_id'].iloc[1]
    id_atual = df['car_id'].iloc[0]
    tempo = []

    for i in range(len(df)):
        try:
            id_prox = df['car_id'].iloc[i+1]
            id_atual = df['car_id'].iloc[i]
        except:
            pass

        # Se irá mudar de id então é registrado o somatório dos intervalos de tempo estacionado
        # i == len(df)-1 para não pular o ultimo
        if (id_prox != id_atual or i == len(df)-1):
            #ver se tem mais de uma em um dia e contar só uma
            hour = datetime.datetime.fromtimestamp(df['start'].iloc[i]).hour
            auxHour = [0] * 24
            auxHour[hour] += 1
            tempo.append(auxHour)
            
            #Somando todas a quantidade de ocorrências de cada horário
            tempo = pd.DataFrame(tempo)
            # Dividindo pelo numero de dias
            # fazer isso nas estações
            tempo = list(pd.Series.sum(tempo))
            
            tempo = [id_atual] + tempo
            
            df_cont.append(tempo)
            tempo = []
        else:
            # Verificando a hora de inicio da viagem e somando em uma lista que representa as 24h
            hour = datetime.datetime.fromtimestamp(df['start'].iloc[i]).hour
            auxHour = [0] * 24
            auxHour[hour] += 1
            #Armazenando a quantidade de viagens em dada hora
            tempo.append(auxHour)
    
    #Labels das colunas
    labels = list(range(-1,24))
    [format(x,'02d') for x in labels]
    labels[0] = 'car_id'
    
    df_cont = pd.DataFrame(df_cont, columns=labels)
    df_cont = df_cont.sort_values(by=['car_id'])
    
    return df_cont

In [40]:
hours = cont_travels(dfTravels)

In [41]:
# Preparando o dataframe para colocar a location, latitude e longitude
lat = [0]*len(hours)
lon = [0]*len(hours)
location = [0]*len(hours)

lat = pd.DataFrame(lat, columns=['lat'])
lon = pd.DataFrame(lon, columns=['lon'])
location = pd.DataFrame(location, columns=['location'])

hours = pd.concat([hours, location, lat, lon], axis=1)

In [ ]:
# Colocando as coordenadas de cada carro
for i in range(len(hours)):
    try:
        coord = car_coord[car_coord['car_id'] == hours['car_id'].iloc[i]]
        hours['lat'].iloc[i] = coord['lat'].iloc[0]
        hours['lon'].iloc[i] = coord['lon'].iloc[0]
        hours['location'].iloc[i] = coord['location'].iloc[0]
    except Exception as e:
        # Carros que sairam da frota
        print(e)
        print('id:'+str(hours['car_id'].iloc[i]))

hours = hours.sort_values(by='location')

In [ ]:
# Somando todos os valores para cada estação
while(len(hours) > 456):
    for i in range(len(hours)):
        try:
            if (hours['location'].iloc[i] == hours['location'].iloc[i+1]):
                print('Antes:')
                print(len(hours))
                for j in range(24):
                    hours[j].iloc[i] = hours[j].iloc[i] + hours[j].iloc[i+1]
                hours = hours.drop(hours.index[i+1])
                hours.index = range(len(hours))
                print('Depois:')
                print(len(hours))
        except:
            break

In [ ]:
# Ordenando por estacionamento
hours = hours.sort_values(by='location')

# Dividindo pela quantidade de dias e numero de carros
for i in range(len(hours)):
    for j in range(24):
        hours[j].iloc[i] = hours[j].iloc[i] / 31

## Normalizando os dados

In [45]:
data = pd.DataFrame()
aux  = pd.DataFrame()

# Organizando todos os dados em uma lista
for i in range(0,24):
    hora = [i] * len(hours)
    hora = pd.DataFrame(hora)
    
    hours.dropna(how='any', axis=0, inplace=True)
    
    aux = pd.concat([hours['car_id'], hours[i], hora], axis=1)
    aux.columns = ['car_id', 'mean', 'hour']
    
    data = data.append(aux)

# Normalizando pelo maior
maior = data['mean'].max()
data['mean'] = data['mean'] / maior

In [46]:
# Reorganizando o dataframe com os dados já normalizados
for i in range(24):
    aux = data[data['hour'] == i]
    hours[i] = aux['mean']

In [47]:
# Ordenando e concatenando os valores com as coordenadas
hours = hours.sort_values(by='car_id')
car_coord = car_coord.sort_values(by='car_id')

aux_csv = hours
aux_csv.dropna(how='any', axis=0, inplace=True)

In [48]:
# Csv's de cada hora
for i in range(24):
    aux_csv[['lat', 'lon', i]].to_csv('ViagensPorHora/Hour'+str(i)+'.csv')

## Plotagem em mapas de calor de cada hora

In [52]:
import geoplotlib as gpl
from geoplotlib.utils import read_csv, BoundingBox, DataAccessObject

In [ ]:
# Imprimindo todas as 24 horas
for i in range(0,24):
    
    hora = str(i)
    
    # Lendo csv com dados de tempo estacionado médio, latitude, longitude de cada estacionamento
    location = pd.read_csv('ViagensPorHora/Hour'+hora+'.csv', usecols=[1,2,3])
    data = location
    # Multiplicando os valores por um escalar para se tornarem mais visíveis
    location[hora] = location[hora] * 100
    location_aux = []
    
    
    # Utilizando um auxiliar para gerar repetições de incidencias para a plotagem no mapa de calor
    for i in range(len(location)):
        for j in range(int(location[hora].iloc[i])):
            location_aux.append([location['lat'].iloc[i], location['lon'].iloc[i]])
    
    location_aux = pd.DataFrame(location_aux, columns=['lat', 'lon'])
    
    # Vancouver    
    
    gpl.kde(location_aux, bw=3, cut_below=1e-4, cmap='jet', alpha=150 )
    data = DataAccessObject(pd.DataFrame({'lat': [],'lon': []}))
    gpl.hist(data, scalemin=0, scalemax=100, cmap='jet', colorscale='lin', alpha=190)
    
    # Coordenadas para o mapa focar em Vancouver
    lat = pd.DataFrame([49.246292, 49.262428, 49.24966])
    lon = pd.DataFrame([-123.11554, -123.116226, -123.04464])
    
    gpl.set_bbox(BoundingBox.from_points(lon[0], lat[0]))
    gpl.request_zoom(12)
    gpl.set_window_size(1280,700)
    gpl.savefig('ViagensPorHora/ViagensPorHoraPNGs/ViagensTodos/vanc_'+hora)
    
    # Victoria
    
    gpl.kde(location_aux, bw=3, cut_below=1e-4, cmap='jet', alpha=150 )
    data = DataAccessObject(pd.DataFrame({'lat': [],'lon': []}))
    gpl.hist(data, scalemin=0, scalemax=100, cmap='jet', colorscale='lin', alpha=190)
    
    # Coordenadas para o mapa focar em Victoria
    lat = pd.DataFrame([48.42666, 48.44344, 48.44560])
    lon = pd.DataFrame([-123.36027,-123.35853,-123.33673])
    
    gpl.set_bbox(BoundingBox.from_points(lon[0], lat[0]))
    gpl.request_zoom(13)
    gpl.set_window_size(1280,700)
    
    gpl.savefig('ViagensPorHora/ViagensPorHoraPNGs/ViagensTodos/vic_'+hora)